<a href="https://colab.research.google.com/github/anandaditya07/Hate-Speech/blob/main/ML_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install -q transformers datasets sentencepiece scikit-learn pandas numpy emoji --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version

1. **Import Libraries**

In [14]:
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

2. **Load Dataset**

In [15]:
train_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-train.csv", encoding='latin-1')
test_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-test.csv", encoding='latin-1')

train_df.head()  #used for previewing the dataset

,tweet,label
0,à¤ à¤®à¤° à¤à¤à¤¾à¤²à¤¾ à¤¦à¥à¤¨à¤¿à¤ à¤¸à...,0
1,à¤¬à¥à¤à¥à¤ªà¥ @BJP4India à¤à¥à¤¨à¥à¤¦à...,0
2,#AssemblyElections2022 à¤ªà¥à¤°à¤­à¤¾à¤°à¥ à...,0
3,#à¤¬à¥à¤°à¥à¤à¤¿à¤à¤ - à¤®à¤§à¥à¤¯à¤ªà¥...,0
4,#AmitShah à¤¨à¥ à¤à¤¹à¤¾ à¤à¤¤à¥à¤¤à¤° à¤ª...,0


**3. Basic Data Inspection**

In [16]:
print(train_df.columns)  #Displaying all column names present in the dataset
print(train_df.isnull().sum())  #Checking missing (null) values in each column
print(train_df['label'].value_counts()) #Counting the number of samples in each class (e.g., hate vs non-hate)

Index(['tweet', 'label'], dtype='object')
tweet    0
label    0
dtype: int64
label
0    16805
1     2214
Name: count, dtype: int64


**4. Text Cleaning Function**

In [17]:
def clean_text(text):
    text = str(text).lower()                     #Converting text to lowercase for consistency.
    text = re.sub(r"http\S+|www\S+", "", text)   # Removing URLs that do not contribute to meaning.
    text = re.sub(r"@\w+|#\w+", "", text)        # Removing mentions and hashtags to reduce noise
    text = emoji.replace_emoji(text, replace="") #Removing emojis that cannot be processed by traditional NLP models.
    text = re.sub(r"[^a-zA-Z\s]", "", text)      #Removing numbers and special characters.
    text = re.sub(r"\s+", " ", text).strip()     #Removing extra spaces and formatting text properly.
    return text

**5. Apply Cleaning**

In [18]:
train_df['clean_text'] = train_df['tweet'].apply(clean_text)
test_df['clean_text'] = test_df['text'].apply(clean_text)

**6. Train–Validation Split**

In [19]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['clean_text'],
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])


Training samples: 15215
Validation samples: 3804


**7. Remove Empty Cleaned Text**

In [20]:
train_df = train_df[train_df['clean_text'].str.strip() != ""]
train_df.reset_index(drop=True, inplace=True)

print("After removing empty rows:", train_df.shape)


After removing empty rows: (3153, 3)


**8. Cell 8: Train–Validation Split**

In [21]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['clean_text'],
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])


Training samples: 2522
Validation samples: 631


**9. Verify Label Distribution**

In [22]:
print("\nTraining labels:\n", y_train.value_counts())
print("\nValidation labels:\n", y_val.value_counts())

display(X_train.head())


Training labels:
 label
0    2276
1     246
Name: count, dtype: int64

Validation labels:
 label
0    570
1     61
Name: count, dtype: int64


,clean_text
2336,what say also amp how are you
2799,l l
2619,sanjay chandra shrestha pratisha joshi
2510,my piece on ecs wrongful steps against moral p...
739,up goa


**BASELINE MODEL (TF-IDF + Logistic Regression)**

**10. TF-IDF Vectorization**

In [23]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

**11. Train Baseline Model**

In [24]:
baseline_model = LogisticRegression(max_iter=1000)
baseline_model.fit(X_train_tfidf, y_train)

y_pred = baseline_model.predict(X_val_tfidf)

print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95       570
           1       0.00      0.00      0.00        61

    accuracy                           0.90       631
   macro avg       0.45      0.50      0.47       631
weighted avg       0.82      0.90      0.86       631



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**12.  Save Baseline Predictions**

In [25]:
test_tfidf = tfidf.transform(test_df['clean_text'])
test_df['baseline_prediction'] = baseline_model.predict(test_tfidf)

test_df.to_csv("baseline_predictions.csv", index=False)
print("Baseline predictions saved")

Baseline predictions saved


**TRANSFORMER MODEL (MuRIL)**

**13. Load MuRIL**

In [26]:
model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels = train_df['label'].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**14. Dataset Class**

In [27]:
class HateDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        item = {k: v.squeeze() for k, v in encoding.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

**15. Create Dataset Objects**

In [28]:
train_dataset = HateDataset(X_train.tolist(), y_train.tolist())
val_dataset = HateDataset(X_val.tolist(), y_val.tolist())

**16. Training Arguments**

In [35]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

**17. Trainer**

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

NameError: name 'training_args' is not defined

**18. Train MuRIL**

In [34]:
trainer.train()

NameError: name 'trainer' is not defined

**19. Evaluate MuRIL**

In [ ]:
preds = trainer.predict(val_dataset)
y_pred_muril = np.argmax(preds.predictions, axis=1)

print(classification_report(y_val, y_pred_muril))
print("Macro F1:", f1_score(y_val, y_pred_muril, average="macro"))


**20. Predict on Test Data**

In [ ]:
test_dataset = HateDataset(test_df['clean_text'].tolist())
test_outputs = trainer.predict(test_dataset)

test_df['muril_prediction'] = np.argmax(test_outputs.predictions, axis=1)
test_df.to_csv("muril_predictions.csv", index=False)

print("MuRIL predictions saved")